This notebook will take the corpus, translate it into a gensim TaggedDocument list, and train the doc2vec model in order to extract the document vecotrs.  We will then translate that into a pandas data frame and save to a file for use in kmeans clustering.

In [1]:
import pickle
import argparse
import pandas as pd
from os import path
from os import listdir
from azureml.core import Dataset, Workspace
#from azureml.data import FileDataset
from tempfile import TemporaryDirectory
import numpy as np
from pprint import pprint
#from azureml.core import Run
from gensim.models import Doc2Vec as d2v
from gensim.models.doc2vec import TaggedDocument as td
import joblib

state_gov_nm = 'MA'
year_to_process = '2020'
data_file_location = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/beefy/code/QuigleyData'
#vector_size = [32, 64, 96, 128, 160, 192]
vector_size = [96]
#state_gov_nm = sys.argv[1]
#year_to_process = sys.argv[2]
#data_file_location = sys.argv[3]

In [2]:
ws = Workspace.get(name="ml-quigley"
                   , subscription_id='8d8bcbfe-86b4-42bb-a470-a8cf97c98d69'
                   , resource_group='project-quigley')

#local_ds_path = tempfile.mkdtemp(prefix='tmp',dir='txt_dataset')

#define the azureml dataset
leg_text_ds = Dataset.get_by_name(ws,'leg_text_ds')

#_embeddings = path.join(data_file_location, state_gov_nm + 'embedded_legdoc.df')

In [3]:
legcorpus = []
# mount dataset onto the mounted_path of a Linux-based compute
#mount_context = leg_text_ds.mount(local_ds_path)

#mount_context.start()
#mount the azureml dataset
#if 1==1:
with TemporaryDirectory() as local_ds_path:
    with leg_text_ds.mount(local_ds_path):
        for f in listdir(local_ds_path):
            if state_gov_nm in f:
                _legcorpus = path.join(local_ds_path, f)
                with open(_legcorpus,'rb') as fl:
                    legcorpus.extend(pickle.load(fl))    

#mount_context.end()
                
print(len(legcorpus))

43482


In [4]:
#legcorpus = [x[] if not isinstance(x[2], list) else x for x in legcorpus]
clean_legcorpus = []
for i, b in enumerate(legcorpus):    
    if not isinstance(b[2],list):
    #    print(i, type(b[2]))
        clean_legcorpus.append([b[0],b[1],[]])
    else:
        clean_legcorpus.append(b)


In [5]:
texts = []
bill_id = []
doc_id = []
for b in clean_legcorpus:
    texts.append(b[2])
    bill_id.append(b[0])
    doc_id.append(b[1])

In [6]:
# here we'll transform the legcorpus into a "tagged" document for use with doc2vec
# the tags will simply be the document ids.  Since we won't be using the predictor part of the operation, and only the resulting weights,
# we don't need to concern ourselves with labeling the documents
tagged_legdocs = []
for l in clean_legcorpus:
    #if i == 1:
        #print(td(l[2], [l[0]]))
#use doc ids
#    tagged_legdocs.append(td(l[2], [str(l[1])]))
#use bill_ids
    tagged_legdocs.append(td(l[2], [str(l[0])]))

In [7]:
#loop through different vector lengths and deposit each into a separate file
for v in vector_size:
    embedder = d2v(tagged_legdocs, dm=1, alpha=.025,vector_size=v, min_alpha=.025, min_count=0, epochs=20, workers=16)
    embedder.train(tagged_legdocs, total_examples=embedder.corpus_count, epochs=embedder.epochs)
    print(len(embedder.docvecs))
    print(type(embedder.docvecs))
    # translate the embeddings and tags into a dataframe
    doc_embedding_list = []
    #for i in range(0,9):
    for i in range(len(embedder.docvecs)):
        #print(embedder.docvecs.index_to_doctag(i))
        doc_tag = embedder.docvecs.index_to_doctag(i)
        doc_vec = [doc_tag]
        doc_vec.extend(embedder.docvecs[doc_tag].tolist())
        #print(doc_vec)
        #doc_vec = doc_vec.tolist()

        doc_embedding_list.append(doc_vec)

    #DFlabels = ['doc_id']
    DFlabels = ['bill_id']
    
    for i in range(1, v + 1):
        DFlabels.append('dv' + str(i))

    print(DFlabels)

    doc_embed_DF = pd.DataFrame(doc_embedding_list, columns=DFlabels)
    
    _embeddings = path.join(data_file_location, state_gov_nm + str(v) + 'embedded_legdoc.df')
    
    #dump data out to a binary file
    with open(_embeddings,'wb') as f:
        pickle.dump(doc_embed_DF,f)
    
    #export the model
    _model = path.join(data_file_location, 'model/' + state_gov_nm + '/' + state_gov_nm + str(v) + '_embed.d2v')
    joblib.dump(embedder, _model)

43363
<class 'gensim.models.keyedvectors.Doc2VecKeyedVectors'>
['bill_id', 'dv1', 'dv2', 'dv3', 'dv4', 'dv5', 'dv6', 'dv7', 'dv8', 'dv9', 'dv10', 'dv11', 'dv12', 'dv13', 'dv14', 'dv15', 'dv16', 'dv17', 'dv18', 'dv19', 'dv20', 'dv21', 'dv22', 'dv23', 'dv24', 'dv25', 'dv26', 'dv27', 'dv28', 'dv29', 'dv30', 'dv31', 'dv32', 'dv33', 'dv34', 'dv35', 'dv36', 'dv37', 'dv38', 'dv39', 'dv40', 'dv41', 'dv42', 'dv43', 'dv44', 'dv45', 'dv46', 'dv47', 'dv48', 'dv49', 'dv50', 'dv51', 'dv52', 'dv53', 'dv54', 'dv55', 'dv56', 'dv57', 'dv58', 'dv59', 'dv60', 'dv61', 'dv62', 'dv63', 'dv64', 'dv65', 'dv66', 'dv67', 'dv68', 'dv69', 'dv70', 'dv71', 'dv72', 'dv73', 'dv74', 'dv75', 'dv76', 'dv77', 'dv78', 'dv79', 'dv80', 'dv81', 'dv82', 'dv83', 'dv84', 'dv85', 'dv86', 'dv87', 'dv88', 'dv89', 'dv90', 'dv91', 'dv92', 'dv93', 'dv94', 'dv95', 'dv96']
